In [2]:
from pathlib import Path
from copy import deepcopy
from cjio import cityjson
from shapely.geometry import Polygon
import matplotlib.pyplot as plt
plt.close('all')
from sklearn.preprocessing import FunctionTransformer
from sklearn import cluster
import numpy as np
import geopandas as gpd
from shapely.geometry import Polygon
import os

In [3]:
from shapely.geometry import shape
from functools import partial
import pyproj
from shapely.ops import transform


# define EPSG 7415/EPSG 28992
source_crs = pyproj.CRS("EPSG:7415")
target_crs = pyproj.CRS("EPSG:28992")


source_crs  = pyproj.Proj(init='epsg:7415')
target_crs = pyproj.Proj(init='epsg:28992')


project = pyproj.Transformer.from_crs(source_crs, target_crs, always_xy=True).transform


In [3]:
import os
import json
path = "lod22"
files = os.listdir(path)

gdf = gpd.GeoDataFrame(columns=['building_id', 'surface_type', 'geometry'])


In [ ]:
# convert each cityjson to geojson, NO merging

print(files)
for file in files:
    polygons = []
    surfaces = []
    building_ids = []
    if ".json" not in file:
        continue

    cm = cityjson.load(os.path.join(path,file),transform=True)
    
    tranformation_object = cm.transform
    tranformation_object["scale"] = [0.001,0.001,1]
    
    buildings = cm.get_cityobjects()
    for building_id,data in buildings.items():
        
        if data.type == "Building":
            continue
        #print(data.parents[0])
        #print(data.geometry[0].transform(tranformation_object).boundaries)
        
        for geom in data.geometry:
            
            boundaries = geom.boundaries
            #print(bounda
           
            
            for surface in geom.surfaces:

                idx = np.array(geom.surfaces[surface]["surface_idx"])
                surface_type = geom.surfaces[surface]["type"]
                if idx.any()==None:
                    continue
                #print([Polygon(boundaries[idx[0][0]][index[1]][0]) for index in idx])
                
                smalls = [Polygon(boundaries[index[0]][index[1]][0],boundaries[index[0]][index[1]][1:]) \
                             for index in idx]
               
                
                polygons += smalls
                surfaces += [surface_type]*len(smalls)
                building_ids += [building_id]*len(smalls)

    gdf =  gpd.GeoDataFrame({'building_id': building_ids, 'surface_type': surfaces, 'geometry': polygons},
                             crs=7415)

    gdf["geometry"].set_crs(epsg=7415)
    gdf["geometry"].to_crs(epsg=28992) 

    tile_path = file.split(".")[0] + ".geojson"
    gdf.to_file(os.path.join(path,tile_path), driver='GeoJSON')   
    print("done", tile_path)

In [ ]:
gdf.head(5)

In [ ]:
# merge and convert each cityjson to ONE geojson

print(files)
polygons = []
surfaces = []
building_ids = []
for file in files:
    if ".json" not in file:
        continue
    
    print(file)
    cm = cityjson.load(os.path.join(path,file),transform=True)
    
    tranformation_object = cm.transform
    tranformation_object["scale"] = [0.001,0.001,1]
    
    buildings = cm.get_cityobjects()
    for building_id,data in buildings.items():
        
        if data.type == "Building":
            continue
        #print(data.parents[0])
        #print(data.geometry[0].transform(tranformation_object).boundaries)
        
        for geom in data.geometry:
            
            boundaries = geom.boundaries
            #print(bounda
           
            
            for surface in geom.surfaces:

                idx = np.array(geom.surfaces[surface]["surface_idx"])
                surface_type = geom.surfaces[surface]["type"]
                if idx.any()==None:
                    continue
                #print([Polygon(boundaries[idx[0][0]][index[1]][0]) for index in idx])
                
                smalls = [Polygon(boundaries[index[0]][index[1]][0],boundaries[index[0]][index[1]][1:]) \
                             for index in idx]
               
                
                polygons += smalls
                surfaces += [surface_type]*len(smalls)
                building_ids += [building_id]*len(smalls)

    
gdf =  gpd.GeoDataFrame({'building_id': building_ids, 'surface_type': surfaces, 'geometry': polygons},
                             crs=7415)

gdf["geometry"].set_crs(epsg=7415)
gdf["geometry"].to_crs(epsg=28992)
merge_folder = path + '\merge'
merged_geojson = path + "_merged_building_surfaces.geojson"
gdf.to_file(os.path.join(merge_folder,merged_geojson), driver='GeoJSON')


In [ ]:
### Berlin data is werid, the nested level inside 'boundaries' is 3 or 4, 
### and "BuildingPart", "BuildingInstallation" types exists
# convert Berlin cityjson to geojson
import os
import json
path = "berlin"
files = os.listdir(path)
file = "Berlin_small.json"

# tackle the different nested level inside 'boundaries'
def count_list_levels(lst):
    if not isinstance(lst, list):
        return 0
    if not lst:
        return 1
    return 1 + max(count_list_levels(item) for item in lst)


gdf = gpd.GeoDataFrame(columns=['building_id', 'surface_type', 'geometry'])

'''
for file in files:
    polygons = []
    surfaces = []
    building_ids = []
    types = []
    if ".json" not in file:
        continue
'''
polygons = []
surfaces = []
building_ids = []
parents_ids = {}
cm = cityjson.load(os.path.join(path,file),transform=True)
print(file)
tranformation_object = cm.transform
tranformation_object["scale"] = [0.001,0.001,1]

n_buildings = 0
buildings = cm.get_cityobjects()
for building_id,data in buildings.items():
    '''
    if data.type in types:
        continue
    types.append(data.type)
    '''
    '''data.type == "BuildingInstallation", has 1 element in Berlin_mitte.json'''
    if data.type == "BuildingInstallation":
        print(file, data.type, building_id)
        continue
    
    if data.type == "Building":
        n_buildings += 1
    
        for geom in data.geometry:
            boundaries = geom.boundaries
            # check if the boundaries level (nest lists) is 3 or 4: 
            levels = count_list_levels(boundaries)
            
            for surface in geom.surfaces:
                idx = np.array(geom.surfaces[surface]["surface_idx"])
                surface_type = geom.surfaces[surface]["type"]
                if idx.any()==None:
                    continue

                if levels == 3:
                    #print(f'nest:{levels}, n:{n_buildings}, id:{building_id}')
                    #print(idx)
                    #print(boundaries)
                    ####print([Polygon(boundaries[idx[0][0]][index[1]][0]) for index in idx])
                    
                    smalls = [Polygon(boundaries[index[0]][0],boundaries[index[0]][1:]) for index in idx]
                    
                if levels == 4:
                    #print(f'nest:{levels}, n:{n_buildings}, id:{building_id}')
                    #print(idx)
                    #print(boundaries)
                    smalls = [Polygon(boundaries[index[0]][index[1]][0],boundaries[index[0]][index[1]][1:]) for index in idx]

                polygons += smalls
                surfaces += [surface_type]*len(smalls)
                building_ids += [building_id]*len(smalls)

        if data.type == 'BuildingPart':
            child_id = building_id
            building_id = data.parents[0]
            if building_id in parents_ids.keys():
                parents_ids[building_id].append(child_id)
            else:
                parents_ids[building_id] = [child_id]

            for geom in data.geometry:
                boundaries = geom.boundaries
                
                for surface in geom.surfaces:

                    idx = np.array(geom.surfaces[surface]["surface_idx"])
                    
                    surface_type = geom.surfaces[surface]["type"]
                    if idx.any()==None:
                        continue
                    #for index in idx:
                        #print(boundaries[index[0]][index[1]][0])
                        #print(boundaries[index[0]][index[1]][1:])

                    smalls = [Polygon(boundaries[index[0]][index[1]][0],boundaries[index[0]][index[1]][1:]) \
                                for index in idx]

                    polygons += smalls
                    surfaces += [surface_type]*len(smalls)
                    building_ids += [building_id]*len(smalls)

#print(len(parents_ids), parents_ids.keys())
for key, value in parents_ids.items():
    print(f'Key: {key}, Length of elements: {len(value)}')

print(f'Building number: {n_buildings + len(parents_ids)}, total number of elements: {len(buildings)}')

gdf =  gpd.GeoDataFrame({'building_id': building_ids, 'surface_type': surfaces, 'geometry': polygons},
                            crs=7415)

gdf["geometry"].set_crs(epsg=7415)
gdf["geometry"].to_crs(epsg=28992) 

tile_path = file.split(".")[0] + ".geojson"
gdf.to_file(os.path.join(path,tile_path), driver='GeoJSON')   
print("done", tile_path)



In [ ]:
gdf.head(20)